## TODO

- memory usage of processing sampling shuffle is *huge*
    - subset processing
    - using collate_fn during training
- using summary feature as input

In [1]:
import os
import h5py
import numpy as np
from memory_profiler import profile
import matplotlib.pyplot as plt
import sys
sys.path.append('../src')

In [ ]:
# check dataset generated for experiment set 0 with model L0
data_path = '../../data/dataset'
!ls $data_path
f1 = h5py.File(data_path + '/dataset_part_0.h5', 'r')
f2 = h5py.File(data_path + '/dataset_part_0_snn.h5', 'r')

In [ ]:
f1['data']['seqC'], f1['data']['theta'], f1['data']['probR']
f2['data']['seqC'], f2['data']['theta'], f2['data']['probR']

check if the two datasets are the same (using same seed on different machines)

In [ ]:
print('comparing snn and uzh generated datasets...')
issame_seqC = not np.sum(np.nan_to_num(f1['data']['seqC'][:], -1)!=np.nan_to_num(f2['data']['seqC'][:], -1))
issame_theta = not np.sum(np.nan_to_num(f1['data']['theta'][:], -1)!=np.nan_to_num(f2['data']['theta'][:], -1))
issame_probR = not np.sum(np.nan_to_num(f1['data']['probR'][:], -1)!=np.nan_to_num(f2['data']['probR'][:], -1))
print("seqC are the same") if issame_seqC else print("!seqC are different!")
print("theta are the same") if issame_theta else print("!theta are different!")
print("probR are the same") if issame_probR else print("!probR are different!")

In [ ]:
f1.close()
f2.close()

## merge datasets into one

In [ ]:
# list the files in the data directory using python
data_dir = '../../data/dataset'
files = [f for f in os.listdir(data_dir) if os.path.isfile(os.path.join(data_dir, f))]

data_paths = []
data_file_idxs = []
for data_file in files:
    data_path = f'{data_dir}/{data_file}'
    data_paths.append(data_path)
    # sort the data paths according to the file name
    data_paths.sort(key=lambda x: x.split('_part_')[-1].split('.')[0])
    data_file_idxs.append(int(data_path.split('_part_')[-1].split('.')[0]))

In [ ]:
# create a new h5 file to store the combined dataset
merged_data_path = f'{data_dir}/dataset_L0_exp_set_0.h5'

with h5py.File(merged_data_path, 'w') as merged_data_file:
    
    for data_path, data_file_idx in zip(*(data_paths, data_file_idxs)):
        print(f'adding {data_path} to {merged_data_path}')
        data_file = h5py.File(data_path, 'r')
        # copy the data group from the data file to the merged data file
        set_group = merged_data_file.create_group(f'set_{data_file_idx}')
        data_file.copy('data/seqC', set_group)
        data_file.copy('data/probR', set_group)
        data_file.copy('data/theta', set_group)
        data_file.close()
        merged_data_file.flush()

    print(f'\nmerged file has keys: {list(merged_data_file.keys())}')    
    print(f"\nin one set_0 it has keys: {list(merged_data_file['set_0'].keys())}")
    print(f"seqC has a shape of: {merged_data_file['set_0']['seqC'].shape}")
    print(f"theta has a shape of: {merged_data_file['set_0']['theta'].shape}")
    print(f"probR has a shape of: {merged_data_file['set_0']['probR'].shape}")

# merged_data_file.close()

## process the x with MEM check 

In [2]:
%load_ext memory_profiler

In [7]:
@profile(precision=4)
def load_dataset():    
    data_path = '../../data/dataset/dataset_L0_exp_set_0.h5'
    f = h5py.File(data_path, 'r')
    seqC = f['set_0']['seqC'][:]
    probR = f['set_0']['probR'][:]
    print(seqC.shape, probR.shape)
    seqC = seqC.reshape((seqC.shape[0]*seqC.shape[1]*seqC.shape[2], seqC.shape[3]))
    probR = probR.reshape((probR.shape[0]*probR.shape[1]*probR.shape[2], probR.shape[3]))
    print(seqC.shape, probR.shape)
    return seqC

In [8]:
%%mprun -f load_dataset

load_dataset()

ERROR: Could not find file /tmp/ipykernel_1584372/3769736698.py
(7, 3, 700, 15) (7, 3, 700, 5000, 1)
(14700, 15) (14700, 5000)



Filename: /home/wehe/anaconda3/envs/sbi/lib/python3.7/site-packages/memory_profiler.py

Line #    Mem usage    Increment  Occurences   Line Contents
  1140    338.2 MiB    338.2 MiB           1               def wrapper(*args, **kwargs):
  1141    338.2 MiB      0.0 MiB           1                   prof = get_prof()
  1142    340.3 MiB      2.1 MiB           1                   val = prof(func)(*args, **kwargs)
  1143    340.3 MiB      0.0 MiB           1                   show_results_bound(prof)
  1144    340.3 MiB      0.0 MiB           1                   return val

In [ ]:
# check reshaped dataset
data_path = '../../data/dataset/dataset_L0_exp_set_0.h5'
f = h5py.File(data_path, 'r')
seqC = f['set_0']['seqC'][:]
probR = f['set_0']['probR'][:]
print(seqC.shape, probR.shape)
seqC_ = seqC.reshape((seqC.shape[0]*seqC.shape[1]*seqC.shape[2], seqC.shape[3]))
probR_ = probR.reshape((probR.shape[0]*probR.shape[1]*probR.shape[2], probR.shape[3]))
print(seqC_.shape, probR_.shape)

## build dataset and dataLoader

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
probR = [0.5, 0.3, 0.1, 0.1]

probR    = torch.tensor(probR).unsqueeze_(dim=0).repeat_interleave(10, dim=0) # (C, D*M*S, 1)
x_choice = torch.bernoulli(probR) # (C, D*M*S, 1)
print(x_choice.shape, probR.shape)
probR = torch.tensor(probR).expand(10, -1, -1)
x_choice = torch.bernoulli(probR) # (C, D*M*S, 1)
print(x_choice.shape, probR.shape)

torch.Size([10, 4]) torch.Size([10, 4])
torch.Size([10, 10, 4]) torch.Size([10, 10, 4])


/home/wehe/anaconda3/envs/sbi/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from train.MyData import MyDataset

class MyDataLoader(DataLoader):
    def __init__(self, dataset, batch_size, C, shuffle=True, pin_memory=False, num_workers=0):
        super().__init__(dataset, batch_size, shuffle=shuffle, collate_fn=self.collate_fn, pin_memory=pin_memory, num_workers=num_workers)
        self.C = C
        
    def collate_fn(self, batch):
        # Process the batch
        x_batches, theta_batches = [], []
        
        # x_batch     = torch.empty((self.C * len(batch), batch[0][0].shape[0], batch[0][0].shape[1]+batch[0][2].shape[1]))
        # theta_batch = torch.empty((self.C * len(batch), batch[0][1].shape[0]))
        # Create tensors for probR and x_seqC outside the loop
        x_seqC_tensor = torch.empty((self.C, batch[0][0].shape[0], batch[0][0].shape[1]))
        probR_tensor = torch.empty((self.C, batch[0][2].shape[0], 1))
        print(len(batch), x_seqC_tensor.shape, probR_tensor.shape)
        
        for i, (seqC, theta, probR) in enumerate(batch): # seqC: (D*M*S, 15), theta: (4,), probR: (D*M*S, 1)
            
            # probR    = torch.tensor(probR).unsqueeze_(dim=0).repeat_interleave(self.C, dim=0) # (C, D*M*S, 1)
            probR_tensor.copy_(torch.tensor(probR).unsqueeze_(dim=0).repeat_interleave(self.C, dim=0)) # (C, D*M*S, 1)
            x_choice = torch.bernoulli(probR_tensor) # (C, D*M*S, 1)
            
            # x_seqC   = torch.tensor(seqC).unsqueeze_(dim=0).repeat_interleave(self.C, dim=0) # (C, D*M*S, 15)
            # x = torch.cat([x_seqC, x_choice], dim=-1)
            x_seqC_tensor.copy_(torch.tensor(seqC).unsqueeze_(dim=0).repeat_interleave(self.C, dim=0)) # (C, D*M*S, 15)
            x = torch.cat([x_seqC_tensor, x_choice], dim=-1)
            
            theta = torch.tensor(theta).unsqueeze_(dim=0).repeat_interleave(self.C, dim=0) # (C, 4)
            
            x_batches.append(x)
            theta_batches.append(theta)
        
        # Concatenate the batched tensors
        x_batch = torch.cat(x_batches, dim=0)
        theta_batch = torch.cat(theta_batches, dim=0)
        
        # Shuffle x along the 2nd axis
        # for i in range(x_batch.shape[0]):
        #     indices = torch.randperm(x_batch.shape[1])
        #     x_batch[i] = x_batch[i][indices]
        x_batch = torch.stack([x_batch[i][torch.randperm(x_batch.shape[1])] for i in range(x_batch.shape[0])])
            
        # Shuffle the batched dataset
        indices     = torch.randperm(x_batch.shape[0])
        x_batch     = x_batch[indices]
        theta_batch = theta_batch[indices]
        
        return torch.tensor(x_batch, dtype=torch.float32), torch.tensor(theta_batch, dtype=torch.float32)    

In [3]:
%load_ext memory_profiler

data_path = "../../data/dataset/dataset_L0_exp_set_0.h5"
batch_size = 200
C = 100

# Create a my dataset using the data path
my_dataset = MyDataset(data_path)

# Create a my data loader using the dataset, batch size, and C
my_dataloader = MyDataLoader(my_dataset, batch_size, C, shuffle=False)

@profile
def load_my_dataset():    
    next(iter(my_dataloader))
    

In [4]:
%%mprun -f load_my_dataset

load_my_dataset()

ERROR: Could not find file /tmp/ipykernel_1591563/4076220988.py
200 torch.Size([100, 14700, 15]) torch.Size([100, 14700, 1])


/home/wehe/anaconda3/envs/sbi/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Filename: /home/wehe/anaconda3/envs/sbi/lib/python3.7/site-packages/memory_profiler.py

Line #    Mem usage    Increment  Occurences   Line Contents
  1140    328.7 MiB    328.7 MiB           1               def wrapper(*args, **kwargs):
  1141    328.7 MiB      0.0 MiB           1                   prof = get_prof()
  1142  18648.8 MiB  18320.1 MiB           1                   val = prof(func)(*args, **kwargs)
  1143  18648.8 MiB      0.0 MiB           1                   show_results_bound(prof)
  1144  18648.8 MiB      0.0 MiB           1                   return val

In [ ]:
data = next(iter(my_dataloader))

In [54]:
data[0].shape, data[1].shape

(torch.Size([2500, 14700, 16]), torch.Size([2500, 4]))